1.In computing, the producer-consumer problem (also known as the bounded-buffer
problem) is a classic example of a multi-process synchronization problem. The problem
describes two processes, the producer and the consumer, which share a common, fixed-size
buffer used as a queue.
* The producer’s job is to generate data, put it into the buffer, and start again.
* At the same time, the consumer is consuming the data (i.e. removing it from the
buffer), one piece at a time.

In [3]:
import threading
import time
import logging
import random
import queue

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

BUF_SIZE = 10
q = queue.Queue(BUF_SIZE)

class ProducerThread(threading.Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
        super(ProducerThread,self).__init__()
        self.target = target
        self.name = name

    def run(self):
        while True:
            if not q.full():
                item = random.randint(1,10)
                q.put(item)
                logging.debug('Putting ' + str(item)  
                              + ' : ' + str(q.qsize()) + ' items in queue')
                time.sleep(random.random())
        return

class ConsumerThread(threading.Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
        super(ConsumerThread,self).__init__()
        self.target = target
        self.name = name
        return

    def run(self):
        while True:
            if not q.empty():
                item = q.get()
                logging.debug('Getting ' + str(item) 
                              + ' : ' + str(q.qsize()) + ' items in queue')
                time.sleep(random.random())
        return

if __name__ == '__main__':
    
    p = ProducerThread(name='producer')
    c = ConsumerThread(name='consumer')

    p.start()
    time.sleep(2)
    c.start()
    time.sleep(2)


(producer ) Putting 10 : 1 items in queue
(producer ) Putting 10 : 2 items in queue
(producer ) Putting 5 : 3 items in queue
(producer ) Putting 9 : 4 items in queue
(producer ) Putting 6 : 5 items in queue
(consumer ) Getting 10 : 4 items in queue
(consumer ) Getting 10 : 3 items in queue
(consumer ) Getting 5 : 2 items in queue
(producer ) Putting 4 : 3 items in queue
(consumer ) Getting 9 : 2 items in queue
(consumer ) Getting 6 : 1 items in queue
(producer ) Putting 8 : 2 items in queue
(consumer ) Getting 4 : 1 items in queue
(producer ) Putting 8 : 2 items in queue
(producer ) Putting 6 : 3 items in queue
(producer ) Putting 1 : 4 items in queue
(consumer ) Getting 8 : 3 items in queue
(consumer ) Getting 8 : 2 items in queue
(producer ) Putting 9 : 3 items in queue
(producer ) Putting 2 : 4 items in queue
(producer ) Putting 7 : 5 items in queue
(producer ) Putting 2 : 6 items in queue
(producer ) Putting 9 : 7 items in queue
(consumer ) Getting 6 : 6 items in queue
(producer ) 

2.Problem Statement – We have a buffer of fixed size. A producer can produce an
item and can place in the buffer. A consumer can pick items and can consume them.
We need to ensure that when a producer is placing an item in the buffer, then at the
same time consumer should not consume any item. In this problem, buffer is the
critical section. To solve this problem, we need two counting semaphores – Full and
Empty. “Full” keeps track of number of items in the buffer at any given time and
“Empty” keeps track of number of unoccupied slots.

In [1]:
import threading
import time
 
# Shared Memory variables
CAPACITY = 10
buffer = [-1 for i in range(CAPACITY)]
in_index = 0
out_index = 0
 
# Declaring Semaphores
mutex = threading.Semaphore()
empty = threading.Semaphore(CAPACITY)
full = threading.Semaphore(0)
 
# Producer Thread Class
class Producer(threading.Thread):
  def run(self):
     
    global CAPACITY, buffer, in_index, out_index
    global mutex, empty, full
     
    items_produced = 0
    counter = 0
     
    while items_produced < 20:
      empty.acquire()
      mutex.acquire()
       
      counter += 1
      buffer[in_index] = counter
      in_index = (in_index + 1)%CAPACITY
      print("Producer produced : ", counter)
       
      mutex.release()
      full.release()
       
      time.sleep(1)
       
      items_produced += 1
 
# Consumer Thread Class
class Consumer(threading.Thread):
  def run(self):
     
    global CAPACITY, buffer, in_index, out_index, counter
    global mutex, empty, full
     
    items_consumed = 0
     
    while items_consumed < 20:
      full.acquire()
      mutex.acquire()
       
      item = buffer[out_index]
      out_index = (out_index + 1)%CAPACITY
      print("Consumer consumed item : ", item)
       
      mutex.release()
      empty.release()      
       
      time.sleep(2.5)
       
      items_consumed += 1
 
# Creating Threads
producer = Producer()
consumer = Consumer()
 
# Starting Threads
consumer.start()
producer.start()
 
# Waiting for threads to complete
producer.join()
consumer.join()

Producer produced :  1
Consumer consumed item :  1
Producer produced :  2
Producer produced :  3
Consumer consumed item :  2
Producer produced :  4
Producer produced :  5
Consumer consumed item :  3
Producer produced :  6
Producer produced :  7
Producer produced :  8
Consumer consumed item :  4
Producer produced :  9
Producer produced :  10
Consumer consumed item :  5
Producer produced :  11
Producer produced :  12
Producer produced :  13
Consumer consumed item :  6
Producer produced :  14
Producer produced :  15
Consumer consumed item :  7
Producer produced :  16
Producer produced :  17
Consumer consumed item :  8
Producer produced :  18
Consumer consumed item :  9
Producer produced :  19
Consumer consumed item :  10
Producer produced :  20
Consumer consumed item :  11
Consumer consumed item :  12
Consumer consumed item :  13
Consumer consumed item :  14
Consumer consumed item :  15
Consumer consumed item :  16
Consumer consumed item :  17
Consumer consumed item :  18
Consumer consume

3.The Dining Philosopher Problem – The Dining Philosopher Problem states
that K philosophers seated around a circular table with one chopstick between
each pair of philosophers. There is one chopstick between each philosopher.
A philosopher may eat if he can pickup the two chopsticks adjacent to him.
One chopstick may be picked up by any one of its adjacent followers but not
both.
There are three states of philosopher : THINKING, HUNGRY and EATING.
Here there are two semaphores : Mutex and a semaphore array for the
philosophers. Mutex is used such that no two philosophers may access the
pickup or putdown at the same time. The array is used to control the behavior
of each philosopher. But, semaphores can result in deadlock due to
programming errors.

In [1]:
import threading
import random
import time

#inheriting threading class in Thread module
class Philosopher(threading.Thread):
    running = True  #used to check if everyone is finished eating

 #Since the subclass overrides the constructor, it must make sure to invoke the base class constructor (Thread.__init__()) before doing anything else to the thread.
    def __init__(self, index, forkOnLeft, forkOnRight):
        threading.Thread.__init__(self)
        self.index = index
        self.forkOnLeft = forkOnLeft
        self.forkOnRight = forkOnRight

    def run(self):
        while(self.running):
            # Philosopher is thinking (but really is sleeping).
            time.sleep(30)
            print ('Philosopher %s is hungry.' % self.index)
            self.dine()

    def dine(self):
        # if both the semaphores(forks) are free, then philosopher will eat
        fork1, fork2 = self.forkOnLeft, self.forkOnRight
        while self.running:
            fork1.acquire() # wait operation on left fork
            locked = fork2.acquire(False) 
            if locked: break #if right fork is not available leave left fork
            fork1.release()
            print ('Philosopher %s swaps forks.' % self.index)
            fork1, fork2 = fork2, fork1
        else:
            return
        self.dining()
        #release both the fork after dining
        fork2.release()
        fork1.release()
 
    def dining(self):			
        print ('Philosopher %s starts eating. '% self.index)
        time.sleep(30)
        print ('Philosopher %s finishes eating and leaves to think.' % self.index)

def main():
    forks = [threading.Semaphore() for n in range(5)] #initialising array of semaphore i.e forks

    #here (i+1)%5 is used to get right and left forks circularly between 1-5
    philosophers= [Philosopher(i, forks[i%5], forks[(i+1)%5])
            for i in range(5)]

    Philosopher.running = True
    for p in philosophers: p.start()
    time.sleep(100)
    Philosopher.running = False
    print ("Now we're finishing.")
 

if __name__ == "__main__":
    main()


Philosopher 3 is hungry.Philosopher 2 is hungry.Philosopher 4 is hungry.
Philosopher 4 starts eating. 

Philosopher 1 is hungry.Philosopher 0 is hungry.Philosopher 2 starts eating. 


Philosopher 1 swaps forks.

Philosopher 2 finishes eating and leaves to think.Philosopher 4 finishes eating and leaves to think.

Philosopher 3 starts eating. Philosopher 1 starts eating. 

Philosopher 0 swaps forks.
Philosopher 3 finishes eating and leaves to think.Philosopher 2 is hungry.Philosopher 4 is hungry.Philosopher 1 finishes eating and leaves to think.



Philosopher 0 starts eating. Philosopher 2 starts eating. 
Philosopher 4 swaps forks.

Now we're finishing.
Philosopher 0 finishes eating and leaves to think.Philosopher 1 is hungry.
Philosopher 3 is hungry.

Philosopher 2 finishes eating and leaves to think.Philosopher 4 starts eating. 

Philosopher 4 finishes eating and leaves to think.


4. Consider a situation where we have a file shared between many people.
* If one of the people tries editing the file, no other person should be reading or
writing at the same time, otherwise changes will not be visible to him/her.
* However if some person is reading the file, then others may read it at the
same time. Precisely in OS we call this situation as the readers-writers
problem
Problem parameters:
>One set of data is shared among a number of processes
>Once a writer is ready, it performs its write. Only one writer may write
at a time
>If a process is writing, no other process can read it
>If at least one reader is reading, no other process can write
>Readers may not write and only read

In [2]:
import threading as thread
import random

global x                #Shared Data
x = 0
lock = thread.Lock()    #Lock for synchronising access

def Reader():
    global x
    print('Reader is Reading!')
    lock.acquire()      #Acquire the lock before Reading (mutex approach)
    print('Shared Data:', x)
    lock.release()      #Release the lock after Reading
    print()

def Writer():
    global x
    print('Writer is Writing!')
    lock.acquire()      #Acquire the lock before Writing
    x += 1              #Write on the shared memory
    print('Writer is Releasing the lock!')
    lock.release()      #Release the lock after Writing
    print()

if __name__ == '__main__':
    for i in range(0, 10):
        randomNumber = random.randint(0, 100)   #Generate a Random number between 0 to 100
        if(randomNumber > 50):
            Thread1 = thread.Thread(target = Reader)
            Thread1.start()
        else:
            Thread2 = thread.Thread(target = Writer)
            Thread2.start()

Thread1.join()
Thread2.join()

Writer is Writing!
Writer is Releasing the lock!

Writer is Writing!
Writer is Releasing the lock!

Writer is Writing!
Writer is Releasing the lock!

Reader is Reading!
Shared Data: 3

Reader is Reading!
Shared Data: 3

Reader is Reading!
Shared Data: 3

Writer is Writing!
Writer is Releasing the lock!

Writer is Writing!
Writer is Releasing the lock!

Reader is Reading!
Shared Data: 5

Reader is Reading!
Shared Data: 5

